### Importaciones y Configuración de AWS

In [1]:
import os
import boto3
import json
import time
import uuid
import requests
from typing import TypedDict, List
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import StateGraph, END


from dotenv import load_dotenv
load_dotenv()


aws_region = os.environ.get("AWS_DEFAULT_REGION")
if not aws_region:
    print("Advertencia: AWS_DEFAULT_REGION no está configurada. Usando la región por defecto del SDK.")

s3_client = None
transcribe_client = None
bedrock_runtime_client = None

try:
    s3_client = boto3.client('s3', region_name=aws_region)
    transcribe_client = boto3.client('transcribe', region_name=aws_region)
    bedrock_runtime_client = boto3.client(
        service_name="bedrock-runtime",
        region_name=aws_region
    )
    print(f"Clientes de S3, Transcribe y Bedrock Runtime inicializados para la región: {aws_region or (bedrock_runtime_client.meta.region_name if bedrock_runtime_client else 'No disponible')}")
except Exception as e:
    print(f"Error inicializando clientes de AWS: {e}")
  
# Líneas de verificación 
#if 'bedrock_runtime_client' in globals() and bedrock_runtime_client is not None:
#    print(f"Verificando bedrock_runtime_client después de la Celda 1: {type(bedrock_runtime_client)}")
#else:
#    print("bedrock_runtime_client NO está definido o es None después de la inicialización en Celda 1.")
#
#if 's3_client' in globals() and s3_client is not None:
#    print(f"Verificando s3_client después de la Celda 1: {type(s3_client)}")
#else:
#    print("s3_client NO está definido o es None después de la inicialización en Celda 1.")

Clientes de S3, Transcribe y Bedrock Runtime inicializados para la región: us-east-1


### Definición de la Función para Transcribir Audio

In [2]:
def transcribe_audio_file(audio_file_path: str, bucket_name: str, job_name_prefix: str = "Nivel2-VoiceAgent-Transcription-"):
    """
    Sube un archivo de audio a S3, inicia un trabajo de transcripción en Amazon Transcribe,
    espera a que se complete y devuelve el texto transcrito.

    Args:
        audio_file_path (str): Ruta local al archivo de audio.
        bucket_name (str): Nombre del bucket de S3 donde se subirá el audio.
        job_name_prefix (str): Prefijo para el nombre del trabajo de transcripción.

    Returns:
        str: El texto transcrito, o None si ocurre un error.
    """
    if not os.path.exists(audio_file_path):
        print(f"Error: El archivo de audio no se encuentra en '{audio_file_path}'")
        return None

    file_name = os.path.basename(audio_file_path)
    unique_id = str(uuid.uuid4())
    s3_object_key = f"transcribe-input/{unique_id}-{file_name}"
    transcription_job_name = f"{job_name_prefix}{unique_id}"

    try:
        print(f"Subiendo '{audio_file_path}' a S3 bucket '{bucket_name}' como '{s3_object_key}'...")
        s3_client.upload_file(audio_file_path, bucket_name, s3_object_key)
        media_file_uri = f"s3://{bucket_name}/{s3_object_key}"
        print(f"Archivo subido a: {media_file_uri}")

        file_format = file_name.split('.')[-1].lower()
        if file_format not in ['mp3', 'mp4', 'wav', 'flac', 'ogg', 'amr', 'webm']:
            print(f"Error: Formato de archivo '{file_format}' no soportado directamente por Transcribe. Por favor usa mp3, mp4, wav, flac, ogg, amr, webm.")
            return None

        print(f"Iniciando trabajo de transcripción '{transcription_job_name}'...")
        transcribe_client.start_transcription_job(
            TranscriptionJobName=transcription_job_name,
            Media={'MediaFileUri': media_file_uri},
            MediaFormat=file_format,
            LanguageCode='es-ES'
            
        )

        while True:
            status = transcribe_client.get_transcription_job(TranscriptionJobName=transcription_job_name)
            job_status = status['TranscriptionJob']['TranscriptionJobStatus']
            if job_status in ['COMPLETED', 'FAILED']:
                print(f"Estado del trabajo de transcripción: {job_status}")
                break
            print(f"Procesando transcripción (estado actual: {job_status})... Esperando 10 segundos.")
            time.sleep(10)

        if job_status == 'FAILED':
            print(f"Error: El trabajo de transcripción falló. Razón: {status['TranscriptionJob'].get('FailureReason')}")
            return None

        transcript_file_uri = status['TranscriptionJob']['Transcript']['TranscriptFileUri']
        print(f"Archivo de transcripción disponible en: {transcript_file_uri}")

        response = requests.get(transcript_file_uri)
        response.raise_for_status() 
        transcript_json = response.json()
        
        transcript_text = transcript_json['results']['transcripts'][0]['transcript']
        print(f"Texto Transcrito: {transcript_text}")
        
        return transcript_text

    except Exception as e:
        print(f"Ocurrió un error durante el proceso de transcripción: {e}")
        return None
    
    finally:
        try:
            print(f"Limpiando: Eliminando objeto '{s3_object_key}' de S3 bucket '{bucket_name}'...")
            s3_client.delete_object(Bucket=bucket_name, Key=s3_object_key)
            print("Objeto S3 eliminado.")
        except Exception as e:
            print(f"Error al eliminar objeto de S3: {e}")


### Definir Variables para la Transcripción

In [3]:
S3_BUCKET_NAME = "chatvoice01" 
LOCAL_AUDIO_FILE_PATH = "../data/audio1.mp3" 


bucket_configured_correctly = False 
audio_file_exists = False 
print(f"Configuración actual:")
print(f"  S3_BUCKET_NAME = '{S3_BUCKET_NAME}'")
print(f"  LOCAL_AUDIO_FILE_PATH = '{LOCAL_AUDIO_FILE_PATH}'")


placeholder_bucket_name = "tu-nombre-de-bucket-s3-aqui"
if not S3_BUCKET_NAME or S3_BUCKET_NAME == placeholder_bucket_name:
    print("\nERROR DE CONFIGURACIÓN:")
    print(f"  POR FAVOR, EDITA LA VARIABLE 'S3_BUCKET_NAME' Y DEFINE EL NOMBRE DE TU BUCKET S3.")
    print(f"  (Valor actual: '{S3_BUCKET_NAME}', Placeholder que se debe cambiar: '{placeholder_bucket_name}')")
else:
    print(f"  Bucket S3 a usar: {S3_BUCKET_NAME} (Parece configurado).")
    bucket_configured_correctly = True


absolute_audio_path = os.path.abspath(LOCAL_AUDIO_FILE_PATH)

if os.path.exists(absolute_audio_path):
    print(f"  Archivo de audio local encontrado en: {absolute_audio_path}")
    audio_file_exists = True
else:
    print("\nERROR DE CONFIGURACIÓN:")
    print(f"  Archivo de audio NO encontrado en la ruta resuelta: '{absolute_audio_path}'")
    print(f"  (Basado en LOCAL_AUDIO_FILE_PATH: '{LOCAL_AUDIO_FILE_PATH}' y directorio actual: '{os.getcwd()}')")
    print(f"  POR FAVOR, ASEGÚRATE DE QUE TU ARCHIVO DE AUDIO EXISTA EN ESA RUTA O AJUSTA 'LOCAL_AUDIO_FILE_PATH'.")

if bucket_configured_correctly and audio_file_exists:
    print("\nConfiguración de variables parece correcta para continuar.")
else:
    print("\nREVISA LOS ERRORES DE CONFIGURACIÓN ANTERIORES ANTES DE CONTINUAR.")



Configuración actual:
  S3_BUCKET_NAME = 'chatvoice01'
  LOCAL_AUDIO_FILE_PATH = '../data/audio1.mp3'
  Bucket S3 a usar: chatvoice01 (Parece configurado).
  Archivo de audio local encontrado en: c:\Users\Sebastian Diaz G\OneDrive\asistente-compras-inteligente\asistente-compras-inteligente\data\audio1.mp3

Configuración de variables parece correcta para continuar.


### Llamar a la función de transcripción y obtener el texto

In [4]:
print("--- Iniciando Proceso de Transcripción del Audio ---")

transcribed_text = None

if S3_BUCKET_NAME and S3_BUCKET_NAME != "tu-nombre-de-bucket-s3-aqui" and \
   LOCAL_AUDIO_FILE_PATH and os.path.exists(os.path.abspath(LOCAL_AUDIO_FILE_PATH)):
    
    print(f"Intentando transcribir el archivo: {os.path.abspath(LOCAL_AUDIO_FILE_PATH)}")
    print(f"Usando el bucket S3: {S3_BUCKET_NAME}")
    

    transcribed_text = transcribe_audio_file(
        audio_file_path=LOCAL_AUDIO_FILE_PATH,
        bucket_name=S3_BUCKET_NAME
    )

    if transcribed_text:
        print("\n--- Transcripción Exitosa ---")
        print(f"Texto Obtenido: \"{transcribed_text}\"")
    else:
        print("\n--- Fallo en la Transcripción ---")
        print("No se pudo obtener el texto transcrito. Revisa los mensajes de error anteriores de la función 'transcribe_audio_file'.")
else:
    print("Error: No se puede proceder con la transcripción.")
    if not S3_BUCKET_NAME or S3_BUCKET_NAME == "tu-nombre-de-bucket-s3-aqui":
        print("  - El nombre del bucket S3 (S3_BUCKET_NAME) no está configurado correctamente.")
    if not LOCAL_AUDIO_FILE_PATH or not os.path.exists(os.path.abspath(LOCAL_AUDIO_FILE_PATH)):
        print("  - La ruta al archivo de audio (LOCAL_AUDIO_FILE_PATH) no es válida o el archivo no existe.")



--- Iniciando Proceso de Transcripción del Audio ---
Intentando transcribir el archivo: c:\Users\Sebastian Diaz G\OneDrive\asistente-compras-inteligente\asistente-compras-inteligente\data\audio1.mp3
Usando el bucket S3: chatvoice01
Subiendo '../data/audio1.mp3' a S3 bucket 'chatvoice01' como 'transcribe-input/727bd084-6422-425b-9c81-9149464c1780-audio1.mp3'...
Archivo subido a: s3://chatvoice01/transcribe-input/727bd084-6422-425b-9c81-9149464c1780-audio1.mp3
Iniciando trabajo de transcripción 'Nivel2-VoiceAgent-Transcription-727bd084-6422-425b-9c81-9149464c1780'...
Procesando transcripción (estado actual: IN_PROGRESS)... Esperando 10 segundos.
Estado del trabajo de transcripción: COMPLETED
Archivo de transcripción disponible en: https://s3.us-east-1.amazonaws.com/aws-transcribe-us-east-1-prod/449814909790/Nivel2-VoiceAgent-Transcription-727bd084-6422-425b-9c81-9149464c1780/22314b1c-ebe9-429e-a56e-fae773d7d104/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEPX%2F%2F%2F%2F%2F%2F%2F%

### Configuración del LLM para el Agente

In [11]:
MODEL_ID_TITAN_EXPRESS = "amazon.titan-text-express-v1"

is_brc_defined_globally = False
global_brc_type = None
global_brc_object = None

if 'bedrock_runtime_client' in globals():
    is_brc_defined_globally = True
    global_brc_type = type(bedrock_runtime_client)
    global_brc_object = bedrock_runtime_client
else:
    print("ANTES del try para llm, bedrock_runtime_client no está en globals().")

try:
    print("--- Dentro del TRY para inicializar llm ---")
    if 'bedrock_runtime_client' not in globals():
        print("ERROR CRÍTICO INESPERADO: bedrock_runtime_client desapareció de globals() al entrar al try.")
        raise NameError("bedrock_runtime_client no encontrado en globals() dentro del try")

    local_brc_for_chatbedrock = bedrock_runtime_client
    
    print(f"  Dentro del TRY, tipo de local_brc_for_chatbedrock (asignado desde global): {type(local_brc_for_chatbedrock)}")
    print(f"  Dentro del TRY, objeto local_brc_for_chatbedrock: {local_brc_for_chatbedrock}")

    llm = ChatBedrock(
        client=local_brc_for_chatbedrock,
        model_id=MODEL_ID_TITAN_EXPRESS,
        model_kwargs={
            "temperature": 0.1,
        }
    )
    print(f"LLM del Agente ({MODEL_ID_TITAN_EXPRESS}) inicializado/verificado exitosamente.")

except NameError as ne:
    print(f"ERROR (NameError) DENTRO DEL TRY para llm: {ne}")
    print("  Esto sugiere que la variable referenciada no estaba definida en el momento del error.")
    print(f"  Verificación de 'bedrock_runtime_client' en globals() DENTRO del except NameError: {'bedrock_runtime_client' in globals()}")
    if 'bedrock_runtime_client' in globals():
        print(f"    Tipo de bedrock_runtime_client en globals() (en except NameError): {type(bedrock_runtime_client)}")
    llm = None

except Exception as e:  # Capturar otras excepciones
    print(f"ERROR (Otro tipo) DENTRO DEL TRY para llm: {type(e).__name__} - {e}")
    print(f"  Error inicializando ChatBedrock para el agente con {MODEL_ID_TITAN_EXPRESS}.")
    llm = None

--- Dentro del TRY para inicializar llm ---
  Dentro del TRY, tipo de local_brc_for_chatbedrock (asignado desde global): <class 'botocore.client.BedrockRuntime'>
  Dentro del TRY, objeto local_brc_for_chatbedrock: <botocore.client.BedrockRuntime object at 0x0000019E171F5310>
LLM del Agente (amazon.titan-text-express-v1) inicializado/verificado exitosamente.


### Definición del Estado del Grafo (AgentState)

In [12]:

class AgentState(TypedDict):
    userInput: str
    intent: str
    entities: dict
    catalogQueryResult: List[dict]
    finalResponse: str
    callLog: List[str]

### Interpretar Entrada del Usuario (NLU) 

In [ ]:
def interpret_user_input(state: AgentState):
    """
    Toma la entrada del usuario y usa el LLM para extraer la intención y las entidades.
    Prompt refinado para mejorar la extracción de marcas.
    """
    print("---AGENTE NODO: Interpretando Entrada del Usuario (con Titan)---")
    user_input = state["userInput"]
    current_call_log = state.get("callLog", [])

    
    prompt_template = ChatPromptTemplate.from_messages([
        SystemMessage(
            content=(
                "tu única función es analizar la consulta del usuario y devolver únicamente un objeto json válido. "
                "no incluyas ningún texto explicativo, saludo, comentario, o cualquier otra cosa fuera del objeto json. "
                "El objeto JSON debe tener exactamente dos claves de nivel superior: 'intent' (string) y 'entities' (un diccionario). "
                "Las intenciones posibles son: 'buscar_producto', 'comparar_productos', 'pedir_recomendacion', 'ver_carrito', 'saludar', 'despedirse', 'otra'. "
                "Las entidades comunes son: 'nombre_producto', 'marca', 'categoria', 'color', 'talla', 'precio_maximo', 'caracteristicas_adicionales'. "
                "Para la entidad 'marca', si el usuario menciona un nombre específico junto al tipo de producto (ej. 'televisor SuperVision'), considera ese nombre como la marca, incluso si no es una marca globalmente conocida. Extrae el nombre tal cual lo dice el usuario para la marca. "
                "Si una entidad no se encuentra, no la incluyas en el diccionario 'entities'. "
                "Si la intención no es clara o no hay entidades, usa 'otra' y un diccionario 'entities' vacío. "
                "Ejemplo 1 de la ÚNICA salida que debes producir: "
                "{\"intent\": \"buscar_producto\", \"entities\": {\"categoria\": \"televisor\", \"marca\": \"SuperVision\", \"tamaño\": \"55 pulgadas\"}}\n"
                "Ejemplo 2 de la ÚNICA salida que debes producir (si la marca es común): "
                "{\"intent\": \"buscar_producto\", \"entities\": {\"categoria\": \"laptop\", \"marca\": \"Dell\"}}\n"
                "REPITO: Tu respuesta DEBE SER SOLO EL OBJETO JSON y nada más."
            )
        ),
        HumanMessage(content=f"Analiza esta consulta: {user_input}")
    ])

    response_content = "" 
    try:
        if 'llm' not in globals() or not isinstance(llm, ChatBedrock):
            print("ERROR CRÍTICO: La variable 'llm' (ChatBedrock) no está definida o no es del tipo correcto.")
            print("Asegúrate de haber ejecutado la Celda 5 (Configuración del LLM para el Agente) correctamente.")
            current_call_log.append(f"AGENTE_NLU_ERROR: LLM no configurado.")
            return {"intent": "error_nlu_setup", "entities": {}, "callLog": current_call_log}

        formatted_prompt = prompt_template.format_messages(userInput=user_input)
        ai_response = llm.invoke(formatted_prompt)
        response_content = ai_response.content.strip()
        print(f"Respuesta cruda del LLM para NLU (Agente-Titan, prompt refinado): {response_content}")

        json_start_index = response_content.find('{')
        json_end_index = response_content.rfind('}')

        if json_start_index != -1 and json_end_index != -1 and json_end_index > json_start_index:
            potential_json_string = response_content[json_start_index : json_end_index + 1]
            print(f"String JSON potencial extraído (Agente, prompt refinado): {potential_json_string}")
            try:
                parsed_response = json.loads(potential_json_string)
            except json.JSONDecodeError: # Si falla el string extraído, intentar con el original
                print(f"Fallo al parsear JSON extraído, intentando con original: '{response_content}'")
                parsed_response = json.loads(response_content)
        else:
            print(f"No se encontraron delimitadores JSON claros, intentando con original: '{response_content}'")
            parsed_response = json.loads(response_content)

        intent = parsed_response.get("intent", "otra")
        entities = parsed_response.get("entities", {})

        current_call_log.append(f"AGENTE_NLU: Input='{user_input}', Intent='{intent}', Entities='{json.dumps(entities)}', LLM_Raw_Output='{response_content[:100]}...'")
        print(f"Intención extraída (Agente, prompt refinado): {intent}")
        print(f"Entidades extraídas (Agente, prompt refinado): {entities}")
        return {"intent": intent, "entities": entities, "callLog": current_call_log}

    except json.JSONDecodeError as e:
        print(f"Error FINAL al decodificar la respuesta JSON del LLM (Agente-Titan, prompt refinado): {e}")
        print(f"Respuesta del LLM que causó el error: {response_content}")
        current_call_log.append(f"AGENTE_NLU_ERROR: JSONDecodeError. LLM_Raw_Output='{response_content[:100]}...'")
        return {"intent": "error_nlu_format", "entities": {}, "callLog": current_call_log}
    except Exception as e:
        print(f"Error inesperado durante la NLU (Agente-Titan, prompt refinado): {e}")
        current_call_log.append(f"AGENTE_NLU_ERROR: Exception - {str(e)}")
        return {"intent": "error_nlu_unexpected", "entities": {}, "callLog": current_call_log}

### Consultar Catálogo

In [14]:
def load_product_database(db_path="../data/products.json"):
    print(f"Intentando cargar DB desde: {os.path.abspath(db_path)}")
    try:
        with open(db_path, 'r', encoding='utf-8') as f:
            database = json.load(f)
        print(f"Base de datos de productos (Agente) cargada desde '{os.path.abspath(db_path)}'")
        return database
    except FileNotFoundError:
        print(f"AGENTE_CATALOG_ERROR: Archivo DB no encontrado en '{os.path.abspath(db_path)}'.")
        return []
    except Exception as e:
        print(f"AGENTE_CATALOG_ERROR: Error cargando DB: {e}")
        return []

def query_product_catalog(state: AgentState):
    print("---AGENTE NODO: Consultando Catálogo de Productos---")
    entities = state.get("entities", {})
    intent = state.get("intent", "")
    current_call_log = state.get("callLog", [])
    product_database = load_product_database()
    if not product_database:
        current_call_log.append("AGENTE_CATALOG_ERROR: DB no cargada o vacía.")
        print("Error: La base de datos de productos (Agente) está vacía o no se pudo cargar.")
        return {"catalogQueryResult": [], "callLog": current_call_log}
    if intent not in ["buscar_producto", "pedir_recomendacion", "comparar_productos"]:
        current_call_log.append(f"AGENTE_CATALOG_SKIP: Intención '{intent}' no requiere consulta.")
        return {"catalogQueryResult": [], "callLog": current_call_log}
    print(f"Consultando catálogo (Agente) con entidades: {entities}")
    results = []
    if not entities:
        results = []
    else:
        for product in product_database:
            match = True
            entity_categoria = entities.get("categoria", "").lower()
            product_categoria = product.get("categoria", "").lower()
            if entity_categoria and entity_categoria not in product_categoria: match = False
            entity_marca = entities.get("marca", "").lower()
            product_marca = product.get("marca", "").lower()
            if entity_marca and entity_marca not in product_marca: match = False
            entity_nombre = entities.get("nombre_producto", "").lower()
            product_nombre = product.get("nombre", "").lower()
            if entity_nombre and entity_nombre not in product_nombre: match = False
            entity_color = entities.get("color", "").lower()
            product_colores = [str(c).lower() for c in product.get("colores", [])]
            if entity_color and product_colores and entity_color not in product_colores: match = False
            entity_talla = str(entities.get("talla", ""))
            product_tallas = [str(t) for t in product.get("tallas_disponibles", [])]
            if entity_talla and product_tallas and entity_talla not in product_tallas: match = False
            if match: results.append(product)
    if not results:
        current_call_log.append(f"AGENTE_CATALOG_QUERY: Entities='{json.dumps(entities)}', Result='No products found'")
    else:
        summary_results = [{"id": p.get("id"), "nombre": p.get("nombre")} for p in results[:3]]
        current_call_log.append(f"AGENTE_CATALOG_QUERY: Entities='{json.dumps(entities)}', Found='{len(results)} items', ExampleResults='{json.dumps(summary_results)}'")
    return {"catalogQueryResult": results, "callLog": current_call_log}


### Generar Respuesta

In [15]:
def generate_response(state: AgentState):
    print("---AGENTE NODO: Generando Respuesta---")
    user_input = state.get("userInput", "")
    intent = state.get("intent", "")
    entities = state.get("entities", {})
    catalog_results = state.get("catalogQueryResult", [])
    current_call_log = state.get("callLog", [])
    context_for_llm = f"Consulta original (puede ser de voz transcrita): '{user_input}'.\n"
    context_for_llm += f"Intención identificada: '{intent}'.\n"
    if entities: context_for_llm += f"Entidades: {json.dumps(entities)}.\n"
    if intent == "error_nlu_format" or intent == "error_nlu_unexpected":
        final_response_text = "Lo siento, tuve problemas para entender tu solicitud de voz. ¿Podrías intentarlo de nuevo?"
        current_call_log.append(f"AGENTE_RESPONSE_GEN: Intent='{intent}', Generated_Response='{final_response_text}'")
        return {"finalResponse": final_response_text, "callLog": current_call_log}
    if intent in ["saludar", "despedirse"] and not catalog_results:
        final_response_text = "¡Hola! Soy tu asistente de compras. ¿En qué puedo ayudarte hoy?" if intent == "saludar" else "¡Hasta luego!"
        current_call_log.append(f"AGENTE_RESPONSE_GEN: Intent='{intent}', Generated_Response='{final_response_text}'")
        return {"finalResponse": final_response_text, "callLog": current_call_log}
    system_message_content = ""
    if not catalog_results:
        if intent in ["buscar_producto", "pedir_recomendacion", "comparar_productos"]:
            context_for_llm += "No se encontraron productos que coincidan con tu búsqueda.\n"
            system_message_content = "Eres un asistente de compras amigable. Informa al usuario que no se encontraron productos para su búsqueda y sugiere que intente una búsqueda diferente."
        else:
            context_for_llm += "No se requirió información del catálogo.\n"
            system_message_content = "Eres un asistente de compras amigable. Responde concisamente."
    else:
        context_for_llm += "Productos encontrados:\n"
        for i, product in enumerate(catalog_results[:3]):
            context_for_llm += f"  P{i+1}: {product.get('nombre', 'N/A')} (Marca: {product.get('marca', 'N/A')}, Precio: ${product.get('precio', 'N/A')})\n"
            if product.get('descripcion'): context_for_llm += f"    Desc: {product.get('descripcion')}\n"
        if len(catalog_results) > 3: context_for_llm += f"  ... y {len(catalog_results) - 3} más.\n"
        system_message_content = "Eres un asistente de compras amigable. Basándote en la consulta y los productos encontrados, genera una respuesta útil."
    prompt_template = ChatPromptTemplate.from_messages([
        SystemMessage(content=system_message_content),
        HumanMessage(content=context_for_llm + "\nGenera una respuesta adecuada.")
    ])
    try:
        formatted_prompt = prompt_template.format_messages()
        ai_response = llm.invoke(formatted_prompt)
        final_response_text = ai_response.content.strip()
        print(f"Respuesta generada por LLM (Agente): {final_response_text}")
        current_call_log.append(f"AGENTE_RESPONSE_GEN: Intent='{intent}', Found='{len(catalog_results)}', Generated='{final_response_text[:100]}...'")
    except Exception as e:
        print(f"Error en generación de respuesta LLM (Agente): {e}")
        current_call_log.append(f"AGENTE_RESPONSE_GEN_ERROR: {str(e)}")
        final_response_text = "Lo siento, tengo problemas para generar una respuesta."
    return {"finalResponse": final_response_text, "callLog": current_call_log}


### Ensamblaje del Grafo LangGraph

In [16]:
agent_workflow = StateGraph(AgentState)
agent_workflow.add_node("nlu_parser_agent", interpret_user_input)
agent_workflow.add_node("catalog_tool_agent", query_product_catalog)
agent_workflow.add_node("response_generator_agent", generate_response)
agent_workflow.set_entry_point("nlu_parser_agent")
agent_workflow.add_edge("nlu_parser_agent", "catalog_tool_agent")
agent_workflow.add_edge("catalog_tool_agent", "response_generator_agent")
agent_workflow.add_edge("response_generator_agent", END)

try:
    agent_app = agent_workflow.compile()
    print("Grafo del Agente LangGraph compilado exitosamente para Nivel 2.")
except Exception as e:
    print(f"Error al compilar el grafo del Agente LangGraph: {e}")
    agent_app = None

Grafo del Agente LangGraph compilado exitosamente para Nivel 2.


### Ejecutar el Agente con el Texto Transcrito

In [17]:
if agent_app and transcribed_text and llm:
    print("\n--- EJECUTANDO EL AGENTE CON TEXTO TRANSCRITO ---")
    
    agent_initial_input = {"userInput": transcribed_text, "callLog": []}
    print(f"Entrada para el Agente (texto transcrito): \"{transcribed_text}\"")
    
    try:
        agent_final_state = agent_app.invoke(agent_initial_input)

        print("\n--- RESULTADO FINAL DEL AGENTE (NIVEL 2) ---")
        final_agent_response = agent_final_state.get('finalResponse', 'No se generó respuesta final del agente.')
        print(f"Respuesta Final del Agente: {final_agent_response}")
        
        print("\n--- LOG DE LLAMADAS DEL AGENTE (NIVEL 2) ---")
        if agent_final_state.get('callLog'):
            for log_entry in agent_final_state['callLog']:
                print(log_entry)
        else:
            print("El log de llamadas del agente está vacío.")
            
    except Exception as e:
        print(f"Error durante la ejecución del agente del Nivel 2: {e}")

elif not transcribed_text:
    print("No hay texto transcrito para procesar con el agente. Ejecuta la Celda 4 primero.")
elif not agent_app:
    print("El grafo del agente no se compiló correctamente. Revisa la Celda 10.")
elif not llm:
    print("El LLM para el agente no está inicializado. Revisa la Celda 5.")




--- EJECUTANDO EL AGENTE CON TEXTO TRANSCRITO ---
Entrada para el Agente (texto transcrito): "buscó un televisor supervisión de 55 pulgadas."
---AGENTE NODO: Interpretando Entrada del Usuario (con Titan, prompt refinado)---
Respuesta cruda del LLM para NLU (Agente-Titan, prompt refinado): {"intent": "buscar_producto", "entities": {"categoria": "televisor", "marca": "SuperVision", "tamaño": "55 pulgadas"}}
String JSON potencial extraído (Agente, prompt refinado): {"intent": "buscar_producto", "entities": {"categoria": "televisor", "marca": "SuperVision", "tamaño": "55 pulgadas"}}
Intención extraída (Agente, prompt refinado): buscar_producto
Entidades extraídas (Agente, prompt refinado): {'categoria': 'televisor', 'marca': 'SuperVision', 'tamaño': '55 pulgadas'}
---AGENTE NODO: Consultando Catálogo de Productos---
Intentando cargar DB desde: c:\Users\Sebastian Diaz G\OneDrive\asistente-compras-inteligente\asistente-compras-inteligente\data\products.json
Base de datos de productos (Agent